# Advanced Classification Predict

© Explore Data Science Academy

## Honour Code

I **YOUR NAME**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the EDSA honour code (https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.

### Import Libraries and Read In the Data

Do not modify or remove any of the code in these cells.

In [485]:
from IPython.display import Image
from IPython.display import IFrame
from IPython import display
import pandas as pd
import numpy as np

import nltk
import sklearn
import imblearn
import csv
%matplotlib inline

from nltk import TreebankWordTokenizer, SnowballStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import  word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

import string
import urllib
import math
import re

nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mphop\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mphop\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [486]:
train  = pd.read_csv('train.csv')
test = pd.read_csv('test.csv') # no labels

train.head()

,sentiment,message,tweetid
0,1,PolySciMajor EPA chief doesn't think carbon di...,625221
1,1,It's not like we lack evidence of anthropogeni...,126103
2,2,RT @RawStory: Researchers say we have three ye...,698562
3,1,#TodayinMaker# WIRED : 2016 was a pivotal year...,573736
4,1,"RT @SoyNovioDeTodas: It's 2016, and a racist, ...",466954


In [487]:
test.head()

,message,tweetid
0,Europe will now be looking to China to make su...,169760
1,Combine this with the polling of staffers re c...,35326
2,"The scary, unimpeachable evidence that climate...",224985
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928


## Overview

Refer to this diagram to guide you while you build your model. Some of the steps will be fleshed out in this template.

![Overview](process_overview_final.jpg)

## Basic preprocessing

Here is a template you may use for your initial base model. 

### Removing URL's
Write a function that removes URL's from a single tweet. 

**Function input:**
- A single string object (tweet) 

**Function output:**
- The tweet with URL's removed as a single object

In [488]:
# define function that removes URL from single tweet
def remove_urls(text):
    # YOUR CODE HERE 
    return re.sub(r'https?:\/\/.*\/w*\S+','',text)

**Original object:**

In [489]:
tweet = train['message'][2]
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [490]:
remove_urls(tweet)

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late '

**Expected output:**
```python
remove_urls(tweet)

>>'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late  '
```

### Convert to lowercase

Write a function that converts a single tweet to lowercase.

**Function input:**
- A single string object (tweet) 

**Function output:**
- The tweet in lowercase as a single object

In [491]:
def to_lower(text):
    # YOUR CODE HERE 
    return text.lower()

**Original object:**

In [492]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [493]:
to_lower(tweet)

'rt @rawstory: researchers say we have three years to act on climate change before it’s too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'

**Expected output:**

```python 
to_lower(tweet)
>> 'rt @rawstory: researchers say we have three years to act on climate change before it’s too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'
```

### Stemming or lemmatising

Write a function that derives the root words for each of the words in a tweet. You may use stemming _or_ lemmatising here.

**Function input:**
- A single string object (tweet) 

**Function output:**
- The tweet in root words as a _single_ object

In [494]:
def get_roots(text):
    # YOUR CODE HERE 
    lemmatizer = WordNetLemmatizer()
    
    return lemmatizer.lemmatize(text)

**Original object:**

In [495]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [496]:
get_roots(tweet)

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

**Expected output:**
```python
get_roots(tweet)
```
Using `SnowballStemmer()`
```python
>> 'rt @rawstory: research say we have three year to act on climat chang befor it too late https://t.co/wdt0kdur2f https://t.co/z0anpt…'
```

Using `WordNetLemmatizer()`
```python
>> 'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'
```

### Other preprocessing techniques 

Come back here once you have submitted your base model to add any other preprocessing functions yuo wish to add

In [506]:
# YOUR CODE HERE

test['message'] = test['message'].apply(get_roots).apply(to_lower).apply(remove_urls).apply(get_tokens)
test['message'] = test['message'].apply(data_cleaning)
test['message'].head()



0    [ europe ,  will ,  now ,  be ,  looking ,  to...
1    [ combine ,  this ,  with ,  the ,  polling , ...
2    [ the ,  scary ,  , ,  unimpeachable ,  eviden...
3    [ ,  karoli ,  ,  morgfair ,  ,  osborneink , ...
4    [ rt ,  ,  fakewillmoore ,  , " female",  orga...
Name: message, dtype: object

### Tokenising 

Write a function that tokenises a single tweet into a list of tokens.

**Function input:**
- A single string object (tweet) 

**Function output:**
- A list of tokens (objects).

In [497]:
def get_tokens(text):
    # YOUR CODE HERE 
    tokenize = TreebankWordTokenizer()
    return tokenize.tokenize(text)

**Original object:**

In [498]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [499]:
get_tokens(tweet)

['RT',
 '@',
 'RawStory',
 ':',
 'Researchers',
 'say',
 'we',
 'have',
 'three',
 'years',
 'to',
 'act',
 'on',
 'climate',
 'change',
 'before',
 'it’s',
 'too',
 'late',
 'https',
 ':',
 '//t.co/WdT0KdUr2f',
 'https',
 ':',
 '//t.co/Z0ANPT…']

**Expected output:**
```python
get_tokens(tweet)

>> ['RT',
 '@',
 'RawStory',
 ':',
 'Researchers',
 'say',
 'we',
 'have',
 'three',
 'years',
 'to',
 'act',
 'on',
 'climate',
 'change',
 'before',
 'it’s',
 'too',
 'late',
 'https',
 ':',
 '//t.co/WdT0KdUr2f',
 'https',
 ':',
 '//t.co/Z0ANPT…']
```

Besides the preprocessing methods we've already discussed, can you think of how our data can be improved at this stage?

### Vectorising 

Use the code from the webinar to define your vecotiser. For more details on this function have a look at the `sklearn`
<a href="https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html">package documentation on the function</a>. Recall that this function, unlike the others before takes an _**entire column**_ as an argument.

### Putting all the functions together 

Test your cleaning functions on a single tweet. Perform the cleaning procedure on this tweet from start to finish. Make sure the output looks as you would expect. Store the cleaned tweet in `tweet_final`. Don't worry about vectorising for now.

**_Include any other cleaning procedures you add after your base model_**

**Original object:**

In [500]:
tweet

'RT @RawStory: Researchers say we have three years to act on climate change before it’s too late https://t.co/WdT0KdUr2f https://t.co/Z0ANPT…'

In [501]:
# clean tweet from start to finish
tweet1 = remove_urls(tweet)
tweet2 = to_lower(tweet)
tweet3 = get_roots(tweet)
tweet4 = get_tokens(tweet)
tweet_final = get_tokens(remove_urls(to_lower(get_roots(tweet))))
# cleaned tweet
tweet_final

['rt',
 '@',
 'rawstory',
 ':',
 'researchers',
 'say',
 'we',
 'have',
 'three',
 'years',
 'to',
 'act',
 'on',
 'climate',
 'change',
 'before',
 'it’s',
 'too',
 'late']

**Expected output:**
```python
tweet_final
>>['rt',
 '@',
 'rawstory',
 ':',
 'research',
 'say',
 'we',
 'have',
 'three',
 'year',
 'to',
 'act',
 'on',
 'climat',
 'chang',
 'befor',
 'it',
 'too',
 'late']
```

### Preprocessing and cleaning entire column of tweets

Now that we know how to clean a single tweet, we can use `apply()` to clean the entire column in the dataframe. This is what the column looks like to begin with.

In [502]:
train['message'].head()

0    PolySciMajor EPA chief doesn't think carbon di...
1    It's not like we lack evidence of anthropogeni...
2    RT @RawStory: Researchers say we have three ye...
3    #TodayinMaker# WIRED : 2016 was a pivotal year...
4    RT @SoyNovioDeTodas: It's 2016, and a racist, ...
Name: message, dtype: object

Write code that applies your cleaning functions to the entire `message` column of the dataframe using `.apply()`. Do not vectorise just yet. Store this in a new column, `train['message_clean']`. 

In [503]:
# applying cleaning functions to entire column of dataframe (NO VECTORISING)

# YOUR CODE HERE - use apply with all your cleaning functions 

train['message'] = train['message'].apply(get_roots).apply(to_lower).apply(remove_urls).apply(get_tokens)

train['message'].head()

0    [polyscimajor, epa, chief, does, n't, think, c...
1    [it, 's, not, like, we, lack, evidence, of, an...
2    [rt, @, rawstory, :, researchers, say, we, hav...
3    [#, todayinmaker, #, wired, :, 2016, was, a, p...
4    [rt, @, soynoviodetodas, :, it, 's, 2016, ,, a...
Name: message, dtype: object

**Expected output:**

```python
train['message_clean'].head()

>>  0    [polyscimajor, epa, chief, does, n't, think, c...
    1    [it, not, like, we, lack, evid, of, anthropoge...
    2    [rt, @, rawstory, :, research, say, we, have, ...
    3    [#, todayinmaker, #, wire, :, 2016, was, a, pi...
    4    [rt, @, soynoviodetodas, :, it, 2016, ,, and, ...
Name: message_clean, dtype: object
```

In [504]:
# Further data cleaning
def data_cleaning(text):
    text = re.sub(r'#\w*', '',str(text))
    text = re.sub(r'\d+', '',str(text))
    text = re.sub(r'\s\s+', ' ',str(text))
    text = re.sub(r'@[\w]*', '',str(text))
    text = re.sub(r"[.;'""'':@#?!\&/$]+\ *", ' ',str(text))
    text = text.lstrip(' ')
    
    return text


In [505]:
train['message'] = train['message'].apply(data_cleaning)

train.head()

,sentiment,message,tweetid
0,1,"[ polyscimajor , epa , chief , does , ""n t""...",625221
1,1,"[ it , "" s"", not , like , we , lack , evi...",126103
2,2,"[ rt , , rawstory , , researchers , say ,...",698562
3,1,"[ , todayinmaker , , wired , , , was , ...",573736
4,1,"[ rt , , soynoviodetodas , , it , "" s"", ,...",466954


### Vectorising 

Vectorise your cleaned data using the vectoriser you defined earlier. Don't forget to fit the vectoriser to the data you just cleaned. Store your vectorised data in `train_vec`.

In [507]:
# splitting data
X = train['message'].astype(str)
y = train['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.25, random_state = 42)


# YOUR CODE HERE 

## Fitting a simple model on training data and evaluate its performance

Fit a model on your cleaned data. Use `train_vec` as your features. Note that you need to convert `train_vec` to an array. 

You are familiar with this process, so there is less guidance here. You may also refer to train notebooks and the webinar pdf as guidance.

Your basic model should be a logistic regression, however if you try different models you will also perform the following procedure.

1. Split the training data into features and labels.
2. Split the training data further into training and validation data.
3. Fit the model on the training subset. 
4. Predict on the validation subset.
5. Calculate the performance metrics on the validation predictions.
6. Select a model based on validation performance (when you have more than one model).
8. Clean the test data.
9. Predict on the cleaned test data.
7. Write a csv that matches `sample_submission.csv`.
8. Submit to Kaggle.

In [509]:
# vectorising and Model training 
# LogisticRegression

lr = Pipeline([('tfidf',TfidfVectorizer()), ('clf',LogisticRegression(C= 1, class_weight = 'balanced', max_iter = 5000))])
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

RandomForestClassifier

model_rf = Pipeline([('tfidf',TfidfVectorizer()), ('clf', RandomForestClassifier(max_depth = 100, n_estimators = 120))])
model_rf.fit(X_train, y_train)
y_pred_model_rf = model_rf.predict(X_test)

#Support Vector Mechines

model_svc = Pipeline([('tfidf',TfidfVectorizer(max_df = 0.8,min_df = 2,ngram_range=(1,2))), ('svc',SVC(class_weight = 'balanced', max_iter = 5000))])
model_svc.fit(X_train, y_train)
y_pred_model_svc = model_svc.predict(X_test)

# KNN Classifier
model_knn = Pipeline([('tfidf',TfidfVectorizer()),('clf',KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski',p=2))])
model_knn.fit(X_train,y_train)
y_pred_model_knn = model_knn.predict(X_test)

#Naive Bayes
model_nb = Pipeline([('tfidf', TfidfVectorizer()),('clf',MultinomialNB())])
model_nb.fit(X_train,y_train)
y_pred_model_nb = model_nb.predict(X_test)

C:\Users\Mphop\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [510]:
# model evaluation
print('LogisticRegression:\n',metrics.classification_report(y_test,y_pred_lr))
print('RandomForestClassifier:\n',metrics.classification_report(y_test,y_pred_model_rf))
print('SVC model:\n',metrics.classification_report(y_test,y_pred_model_svc))
print('KNN Classifier:\n',metrics.classification_report(y_test,y_pred_model_knn))
print('Naive Bayes:\n',metrics.classification_report(y_test,y_pred_model_knn))


LogisticRegression:
               precision    recall  f1-score   support

          -1       0.53      0.66      0.59       336
           0       0.47      0.55      0.50       547
           1       0.86      0.70      0.78      2178
           2       0.68      0.85      0.75       894

    accuracy                           0.71      3955
   macro avg       0.64      0.69      0.66      3955
weighted avg       0.74      0.71      0.72      3955

RandomForestClassifier:
               precision    recall  f1-score   support

          -1       0.96      0.15      0.25       336
           0       0.71      0.23      0.35       547
           1       0.69      0.92      0.79      2178
           2       0.76      0.71      0.74       894

    accuracy                           0.71      3955
   macro avg       0.78      0.50      0.53      3955
weighted avg       0.73      0.71      0.67      3955

SVC model:
               precision    recall  f1-score   support

          -1     

## Suggestions on how to improve the model - Extras 

Here are some ideas on how you can improve your model. Try some of them out once you have submitted your base model to Kaggle. Only use the methods you find improve your model performance.

 - Additional preprocessing 
     - Use some of the methods in the notebooks to balance the data
     - Replace links with the text `Link` to show the machine that there is a link present without the extraneous detail of the link text
     - Remove punctuation
     - Correct spelling
     - Bag of words vs. N-grams - which is better?
     - Try `TweetTokenizer()` - how does this differ from the tokenisers you are familiar with?
 - Consider which other models could do a better job (refer to course resources)
 - Consider some further feature engineering:
     - What other attributes of the tweets may be useful?
     - Should we perform feature selection given their size and sparseness of the vectorised tweets?
 - If you've mastered the above, try implementing hyperparameter tuning methods.
 - Compare all the models you try and remember to submit **the best one** based on test performance (f1-score using macro averaging). 
 
You may use the space below to try out these "Extras".

## How/why is the model bad?

Do you have any hypotheses as to how you can improve the model further? Use this space to investigate with mindful EDA if necessary.

Be sure to check whether each additional technique you apply actually improves your model performance.

## Submit your best model predictions 

Don't forget to submit your the predictions on the best model to Kaggle.

In [479]:
# write csv for best predictions
test['message'] = test['message'].astype(str)

y_test = model_svc.predict(test['message'])
output = pd.DataFrame({'tweetid':test.tweetid,
                      'sentiment':y_test})
output.to_csv('submission.csv',index = False)


In [480]:
output

,tweetid,sentiment
0,169760,1
1,35326,1
2,224985,1
3,476263,1
4,872928,0
...,...,...
10541,895714,1
10542,875167,1
10543,78329,2
10544,867455,0
